In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt

In [3]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data\FashionMNIST\raw\train-images-idx3-ubyte.gz to data\FashionMNIST\raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data\FashionMNIST\raw\train-labels-idx1-ubyte.gz to data\FashionMNIST\raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting data\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to data\FashionMNIST\raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting data\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to data\FashionMNIST\raw



C:\ProgramData\Anaconda3\lib\site-packages\torchvision\datasets\mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [10]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print("Shape of X [N, C, H, W]: ", X.shape)
    print("Shape of y: ", y.shape, y.dtype)
    break

Shape of X [N, C, H, W]:  torch.Size([64, 1, 28, 28])
Shape of y:  torch.Size([64]) torch.int64


In [11]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [12]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [13]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [14]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [15]:
epochs = 50
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.303911  [    0/60000]
loss: 2.286771  [ 6400/60000]
loss: 2.267155  [12800/60000]
loss: 2.269669  [19200/60000]
loss: 2.243989  [25600/60000]
loss: 2.221242  [32000/60000]
loss: 2.228081  [38400/60000]
loss: 2.194650  [44800/60000]
loss: 2.189980  [51200/60000]
loss: 2.171153  [57600/60000]
Test Error: 
 Accuracy: 44.8%, Avg loss: 2.158740 

Epoch 2
-------------------------------
loss: 2.163016  [    0/60000]
loss: 2.154173  [ 6400/60000]
loss: 2.094731  [12800/60000]
loss: 2.126308  [19200/60000]
loss: 2.068119  [25600/60000]
loss: 2.001227  [32000/60000]
loss: 2.034321  [38400/60000]
loss: 1.951879  [44800/60000]
loss: 1.949926  [51200/60000]
loss: 1.893451  [57600/60000]
Test Error: 
 Accuracy: 57.0%, Avg loss: 1.889616 

Epoch 3
-------------------------------
loss: 1.908546  [    0/60000]
loss: 1.887289  [ 6400/60000]
loss: 1.767754  [12800/60000]
loss: 1.827780  [19200/60000]
loss: 1.713734  [25600/60000]
loss: 1.649969  [32000/600

loss: 0.648978  [ 6400/60000]
loss: 0.438912  [12800/60000]
loss: 0.689373  [19200/60000]
loss: 0.601630  [25600/60000]
loss: 0.582825  [32000/60000]
loss: 0.625785  [38400/60000]
loss: 0.687065  [44800/60000]
loss: 0.655425  [51200/60000]
loss: 0.593810  [57600/60000]
Test Error: 
 Accuracy: 79.2%, Avg loss: 0.601479 

Epoch 23
-------------------------------
loss: 0.519870  [    0/60000]
loss: 0.636926  [ 6400/60000]
loss: 0.429492  [12800/60000]
loss: 0.680418  [19200/60000]
loss: 0.594908  [25600/60000]
loss: 0.576734  [32000/60000]
loss: 0.615031  [38400/60000]
loss: 0.682755  [44800/60000]
loss: 0.651075  [51200/60000]
loss: 0.584136  [57600/60000]
Test Error: 
 Accuracy: 79.5%, Avg loss: 0.593138 

Epoch 24
-------------------------------
loss: 0.508971  [    0/60000]
loss: 0.625784  [ 6400/60000]
loss: 0.420845  [12800/60000]
loss: 0.671937  [19200/60000]
loss: 0.588561  [25600/60000]
loss: 0.571065  [32000/60000]
loss: 0.605094  [38400/60000]
loss: 0.679175  [44800/60000]
loss

loss: 0.514878  [ 6400/60000]
loss: 0.329351  [12800/60000]
loss: 0.566278  [19200/60000]
loss: 0.490141  [25600/60000]
loss: 0.492624  [32000/60000]
loss: 0.504452  [38400/60000]
loss: 0.667314  [44800/60000]
loss: 0.610030  [51200/60000]
loss: 0.454149  [57600/60000]
Test Error: 
 Accuracy: 82.0%, Avg loss: 0.506515 

Epoch 44
-------------------------------
loss: 0.381130  [    0/60000]
loss: 0.511952  [ 6400/60000]
loss: 0.326634  [12800/60000]
loss: 0.562596  [19200/60000]
loss: 0.486071  [25600/60000]
loss: 0.489343  [32000/60000]
loss: 0.501641  [38400/60000]
loss: 0.666538  [44800/60000]
loss: 0.608049  [51200/60000]
loss: 0.450366  [57600/60000]
Test Error: 
 Accuracy: 82.1%, Avg loss: 0.504224 

Epoch 45
-------------------------------
loss: 0.377006  [    0/60000]
loss: 0.509192  [ 6400/60000]
loss: 0.323984  [12800/60000]
loss: 0.559064  [19200/60000]
loss: 0.482193  [25600/60000]
loss: 0.486171  [32000/60000]
loss: 0.498951  [38400/60000]
loss: 0.665613  [44800/60000]
loss

In [16]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [17]:
model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [29]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[10][0], test_data[10][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Coat", Actual: "Coat"


In [22]:
y

9

In [30]:
x

tensor([[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.5686, 0.7451, 0.3804, 0.4275, 0.4118,
          0.7333, 0.2980, 0.0000, 0.0000, 0.0000, 0.0078, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.2510, 0.6667, 0.6392, 0.8157, 0.9529, 0.9412, 0.8980,
          0.6784, 0.6353, 0.6039, 0.0706, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.1529,
          0.6196, 0.7686, 0.5843, 0.7490, 0.7804, 0.8745, 0.9529, 0.7294,
          0.7490, 0.6667, 0.6078, 0.6353, 0.4941, 0.0863, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0118, 0.0000, 0.4118, 0.6314,
          0.5882, 0.5529, 0.5098, 0.7804, 0.9373, 0.8980, 0.8627, 0.7373,
          0.9569, 0.5686, 0.5020, 0.5569, 0.5922, 0.6863,